In [4]:
from skimage.feature import hog
from skimage import color
from skimage.io import imread

import pandas as pd

df = pd.read_csv("./local_path.csv")

#PREFIX = "/data/damoncrockett/"

#df.local_path = df.local_path.map(lambda x: PREFIX + x[33:])

import numpy as np

vector_list = []

smp = 2

for i in range(smp):
    img = imread(df.local_path.loc[i])
    
    if img.shape[0] < 128 or img.shape[1] < 128:
        df.drop(i)
    else:
        img_gray = color.rgb2gray(img)
        fd = hog(img_gray, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(4, 4))
        vector_list.append(fd)
        
X = np.vstack(vector_list)

In [8]:
fd.shape

(24336,)

In [2]:
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
"""
from sklearn.cluster import KMeans

for k in range(2,50):

    kmeans = KMeans(n_clusters = k)
    kmeans.fit(X)

    df = df[:len(vector_list)]

    df['clusters'] = kmeans.labels_

    centroids = pd.DataFrame(kmeans.cluster_centers_)

    from sklearn.decomposition import RandomizedPCA as pca
    pca = pca(n_components=2)
    Y = centroids.as_matrix()
    pca.fit(Y)
    subspace = pd.DataFrame(pca.transform(Y),columns=["x","y"])

    x = subspace.x
    y = subspace.y
    fig, ax = plt.subplots(figsize=(4,4))
    ax.scatter(x, y)

    plt.savefig("./" + str(k) + ".png")
    
    print k, Counter(list(df.clusters))
    """

'\nfrom sklearn.cluster import KMeans\n\nfor k in range(2,50):\n\n    kmeans = KMeans(n_clusters = k)\n    kmeans.fit(X)\n\n    df = df[:len(vector_list)]\n\n    df[\'clusters\'] = kmeans.labels_\n\n    centroids = pd.DataFrame(kmeans.cluster_centers_)\n\n    from sklearn.decomposition import RandomizedPCA as pca\n    pca = pca(n_components=2)\n    Y = centroids.as_matrix()\n    pca.fit(Y)\n    subspace = pd.DataFrame(pca.transform(Y),columns=["x","y"])\n\n    x = subspace.x\n    y = subspace.y\n    fig, ax = plt.subplots(figsize=(4,4))\n    ax.scatter(x, y)\n\n    plt.savefig("./" + str(k) + ".png")\n    \n    print k, Counter(list(df.clusters))\n    '

In [4]:
from PIL import Image
from shapely.geometry import Point
import os

In [7]:
from sklearn.cluster import KMeans

for k in range(7,50):

    os.mkdir("/Users/damoncrockett/Desktop/Leo/"+str(k))
    
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(X)

    df = df[:len(vector_list)]

    df['clusters'] = kmeans.labels_
    centroids = pd.DataFrame(kmeans.cluster_centers_)
    
    euclidean_distance = []

    hog_points = pd.DataFrame(X)

    for i in range(len(df)):
        tmp = hog_points.loc[i].as_matrix()
        cluster_integer = int(df.clusters.loc[i])

        euclidean_distance_i = np.linalg.norm(tmp - centroids.loc[cluster_integer].as_matrix())
        euclidean_distance.append(euclidean_distance_i)

    df['euclidean_distance'] = euclidean_distance
    
    for cluster_label in range(k):
        tmp = df[df.clusters==cluster_label]
        tmp.sort('euclidean_distance',inplace=True)
        tmp.reset_index(drop=True,inplace=True)
        
        n = len(tmp)
        
        grid_side = int(round( np.sqrt(n) / 0.75 ))
        
        x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
        grid_list = pd.DataFrame(x,columns=['x'])
        grid_list['y'] = y

        point = []
        for i in range(len(grid_list)):
            point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

        grid_list['point'] = point

        open_grid = list(grid_list.point)

        midpoint = int(round( grid_side / 2 ))
        centroid_loc = Point(midpoint,midpoint)
        
        if n > 1:
            open_grid.remove(centroid_loc)
        
        thumb_side = 128

        px_w = thumb_side * grid_side
        px_h = thumb_side * grid_side

        canvas = Image.new('RGB',(px_w,px_h),(50,50,50))
        
        for i in range(n):
            im = Image.open(tmp.local_path.loc[i])
            im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
            closest_open = min(open_grid,key=lambda x: centroid_loc.distance(x))
            x = int(closest_open.x) * thumb_side
            y = int(closest_open.y) * thumb_side
            canvas.paste(im,(x,y))
            open_grid.remove(closest_open)
            
        canvas.save("./"+str(k)+"/"+str(cluster_label)+".png")